In [1]:
import io
import requests
from PIL import Image
from torchvision import models, transforms
from torch.autograd import Variable
from torch.nn import functional as F
import numpy as np
import cv2
import torch.nn as nn
import torch
import glob
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import pandas as pd
%matplotlib inline

In [2]:
## Load the trained_model
#azure_chest_xray_sym_test = get_symbol()
model = torch.load("Model_65_50_4-8-18.pth.tar", map_location=lambda storage, loc: storage)
#azure_chest_xray_sym_test.load_state_dict(chkpt['state_dict'])

C:\Users\mansiarora\Anaconda3\lib\site-packages\torch\serialization.py:325: SourceChangeWarning: source code of class 'torch.nn.parallel.data_parallel.DataParallel' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
C:\Users\mansiarora\Anaconda3\lib\site-packages\torch\serialization.py:325: SourceChangeWarning: source code of class 'torch.nn.modules.conv.Conv2d' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
C:\Users\mansiarora\Anaconda3\lib\site-packages\torch\serialization.py:325: SourceChangeWarning: source code of class 'torch.nn.modules.pooling.MaxPool2d' has changed. you can retrieve the original source code by accessing t

In [3]:
def get_symbol(out_features=14, multi_gpu=2):
    model = models.densenet.densenet121(pretrained=True)
    # Replace classifier (FC-1000) with (FC-14)
    model.classifier = nn.Sequential(
        nn.Linear(model.classifier.in_features, out_features))
#         nn.Sigmoid())
    if multi_gpu:
        model = nn.DataParallel(model)
    # CUDA
    #model.cuda()  
    return model

In [4]:
azure_chest_xray_sym = get_symbol()
azure_chest_xray_sym.load_state_dict(model.state_dict())

In [5]:
azure_chest_xray_sym.eval()

DataParallel(
  (module): DenseNet(
    (features): Sequential(
      (conv0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (norm0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True)
      (relu0): ReLU(inplace)
      (pool0): MaxPool2d(kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), dilation=(1, 1), ceil_mode=False)
      (denseblock1): _DenseBlock(
        (denselayer1): _DenseLayer(
          (norm.1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True)
          (relu.1): ReLU(inplace)
          (conv.1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (norm.2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True)
          (relu.2): ReLU(inplace)
          (conv.2): Conv2d(128, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        )
        (denselayer2): _DenseLayer(
          (norm.1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True)
          (relu.1): ReLU(inplace)
          (co

In [6]:
filename = "00005288_014.jpeg"
img_pil = Image.open(filename)
img_pil.thumbnail((224,224),Image.ANTIALIAS)

In [7]:
normalize = transforms.Normalize(
   mean=[0.485, 0.456, 0.406],
   std=[0.229, 0.224, 0.225]
)
preprocess = transforms.Compose([
   transforms.Resize(224),
    transforms.Grayscale(num_output_channels=3),
   transforms.ToTensor(),
   normalize
])

In [8]:
train_img_list = pd.read_pickle("BDH_Project/code/train_val_filtered.pkl")
test_img_list = pd.read_pickle("BDH_Project/code/train_val_filtered.pkl")
label_df = pd.concat([train_img_list[['img_filename', 'text_label']], 
           test_img_list[['img_filename', 'text_label']]])

In [9]:
label_df.drop_duplicates(inplace=True)

In [10]:
label_df.shape

(85690, 2)

In [11]:
label_df[label_df["img_filename"] == "00005288_014.png"]

,img_filename,text_label
16287,00005288_014.png,Edema|Mass|Pneumonia


In [12]:
img_tensor = preprocess(img_pil)

In [13]:
img_variable = Variable(img_tensor.unsqueeze(0))
img_variable.shape

torch.Size([1, 3, 224, 224])

In [14]:
result = azure_chest_xray_sym(img_variable)

In [15]:
result.sum()

Variable containing:
-57.4706
[torch.FloatTensor of size 1]

In [16]:
classes = ['Atelectasis', 'Cardiomegaly', 'Consolidation', 'Edema', 'Effusion', 'Emphysema',
                 'Fibrosis','Hernia', 'Infiltration', 'Mass', 'Nodule', 'Pleural_Thickening',
                 'Pneumonia','Pneumothorax']

In [17]:
result

Variable containing:

Columns 0 to 7 
 -4.5129  -5.6139  -1.9651  -1.4398  -0.5750  -7.7180  -5.8821 -11.1930

Columns 8 to 13 
 -0.3895  -3.8607  -2.4266  -3.4014  -1.6242  -6.8683
[torch.FloatTensor of size 1x14]

In [23]:
h_x = F.softmax(result).data.squeeze()
probs, idx = h_x.sort(0, True)

C:\Users\mansiarora\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  """Entry point for launching an IPython kernel.


In [24]:
probs


 3.4284e-01
 2.8477e-01
 1.1994e-01
 9.9733e-02
 7.0927e-02
 4.4706e-02
 1.6867e-02
 1.0655e-02
 5.5500e-03
 1.8456e-03
 1.4115e-03
 5.2647e-04
 2.2509e-04
 6.9689e-06
[torch.FloatTensor of size 14]

In [25]:
for i in range(0, 14):
    print('{:.3f} -> {}'.format(probs[i], classes[idx[i]]))

0.343 -> Infiltration
0.285 -> Effusion
0.120 -> Edema
0.100 -> Pneumonia
0.071 -> Consolidation
0.045 -> Nodule
0.017 -> Pleural_Thickening
0.011 -> Mass
0.006 -> Atelectasis
0.002 -> Cardiomegaly
0.001 -> Fibrosis
0.001 -> Pneumothorax
0.000 -> Emphysema
0.000 -> Hernia


In [21]:
# Atelectasis|Cardiomegaly|Consolidation|Edema|Effusion|Pleural_Thickening


In [ ]:
# download the imagenet category list
# classes = {int(key):value for (key, value)
#           in requests.get(LABELS_URL).json().items()}
classes = ['Atelectasis', 'Cardiomegaly', 'Consolidation', 'Edema', 'Effusion', 'Emphysema',
                 'Fibrosis','Hernia', 'Infiltration', 'Mass', 'Nodule', 'Pleural_Thickening',
                 'Pneumonia','Pneumothorax']

In [ ]:
h_x = (logit).data.squeeze()
probs, idx = h_x.sort(0, True)

In [ ]:
for i in range(0, 5):
    print('{:.3f} -> {}'.format(probs[i], classes[idx[i]]))